In [1]:
import json
import requests
import popelines
import os
from datetime import datetime, timedelta
from main import fix_values, populating_vals, flatten, flatten_dupe_vals

In [2]:
from generate_original import read_endpoint, generate_projects


In [3]:
from batch_update_projects import check_bq_ts, check_api_ts, generate_new_entity

In [4]:
gbq_key = os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS')

pope = popelines.popeline(dataset_id='optimizely', service_key_file_loc=gbq_key, directory='.', verbose=False)
headers = {
    'Authorization': 'Bearer 2:EWAWmaXb4TgtYVU2VvwoEF-9UbJxBahkiFh1633_Oc9nmju7iJis',
}


# Fuctions

In [ ]:
def check_bq_ts(table_name, ts_col_name):
    last_upload_ts = pope.find_last_entry(table_name, ts_col_name) + timedelta(hours=-8)
    return last_upload_ts

In [ ]:
# this function should get timestamp for each entity (project, experiment, or result), from API,
# returns only those with a timestamp that is later than the benchmark, benchmark_ts
def check_api_ts(api_path, table, ts_col, benchmark_ts, existing_id):
    
    # compare each of the project's last_modified timestamp to the above last upload_ts
    # select only if last_modified > upload_ts
    updating_entity_id = []
    if table == 'project':
        all_existing = generate_projects(api_path, headers)
    # elif table == 'experiment':
    #     all_existing = generate_experiment(api_path, headers)
    else:
        pass

    for entity in all_existing:
        entity_ts = datetime.strptime(entity[ts_col], '%Y-%m-%dT%H:%M:%S.%fz') + timedelta(hours=-8)
        if  entity_ts > benchmark_ts:
            updating_entity_id.append(entity['id'])
        elif entity['id'] not in existing_id:
            updating_entity_id.append(entity['id'])

    return updating_entity_id

In [ ]:
def generate_new_entity(id_list, api_path, table):
    new_json = []
    for entity_id in id_list:
        if table == 'project':
            entity_info = generate_projects(f'{api_path}/{entity_id}', headers)
            entity_info['upload_ts'] = str(datetime.utcnow())

        # need to add experiment table
        else:
            pass

        new_json.append(entity_info)

    return new_json

# Updating new/modified experiments

In [5]:
experiment_endpoint = 'https://api.optimizely.com/v2/experiments'

In [6]:
headers = {
    'Authorization': 'Bearer 2:EWAWmaXb4TgtYVU2VvwoEF-9UbJxBahkiFh1633_Oc9nmju7iJis',
}


In [7]:
params = (
    ('project_id', 9965963792),
    ('per_page', 100),
) 

In [11]:
read_endpoint(endpoint=experiment_endpoint, headers_set=headers, params_set=params)

{'allocation_policy': 'manual',
 'audience_conditions': '["and", {"audience_id": 10562358375}]',
 'campaign_id': 13570680399,
 'changes': [],
 'created': '2019-02-14T22:34:39.931461Z',
 'earliest': '2019-02-20T18:28:36.788491Z',
 'holdback': 0,
 'id': 13559460138,
 'is_classic': False,
 'last_modified': '2019-02-20T18:28:39.647700Z',
 'metrics': [{'aggregator': 'unique',
   'event_id': 13614010314,
   'scope': 'visitor',
   'winning_direction': 'increasing'},
  {'aggregator': 'unique',
   'event_id': 13639340063,
   'scope': 'visitor',
   'winning_direction': 'increasing'},
  {'aggregator': 'bounce',
   'event_id': 13623520170,
   'scope': 'event',
   'winning_direction': 'decreasing'},
  {'aggregator': 'bounce',
   'event_id': 13621520303,
   'scope': 'event',
   'winning_direction': 'decreasing'},
  {'aggregator': 'unique',
   'event_id': 13630910547,
   'scope': 'visitor',
   'winning_direction': 'increasing'},
  {'aggregator': 'unique',
   'event_id': 13642490335,
   'scope': 'visi

In [9]:
response_text = json.loads(response.text)

In [10]:
response_text

{'code': 'invalid_credentials',
 'message': 'Missing authorization header',
 'uuid': '37d9229f-7ae8-4607-8f70-5501f903c5d0'}

In [ ]:
# get the timestamp from bq for experiment table, at which the job was most recently run
last_upload_ts = check_bq_ts('origin_experiments_single_fields', 'upload_ts')



In [ ]:
# get experiments with last_modified timestamps that are later than the previous ts
# getting ready to upload
updating_experiments = check_api_ts(api_path=experiment_endpoint, table='experiment', ts_col='last_modified', benchmark_ts=last_upload_ts.replace(tzinfo=None))
updating_experiments_json = generate_new_entity(id_list=updating_experiments, api_path='https://api.optimizely.com/v2/projects', table='experiment')



In [ ]:
result_endpoint = f'https://api.optimizely.com/v2/experiments/13559460138/timeseries'


In [ ]:
response_ts = requests.get(result_endpoint, headers=headers)